In [16]:
import petl as etl

from pipeline_utils.filesystem.paths import RAW_DATA, SITE

In [17]:
OUT = SITE / 'data/360-giving/_data/'
OUT.mkdir(exist_ok=True, parents=True)

In [36]:
data=(
    etl
    .fromcsv(RAW_DATA / '360-giving.csv')
    .cut(
        'Title',
        'Amount Awarded',
        'Currency',
        'Award Date',

        'Recipient Org:Identifier',
        'Recipient Org:Name',
        'Recipient Org:Charity Number',
        'Recipient Org:Company Number',
        'Recipient Org:Postal Code',

        'Funding Org:Identifier',
        'Funding Org:Name',
        'Funding Org:Postal Code',

        'Grant Programme:Code',
        'Grant Programme:Title',

        'License (see note)',
    )
    .convert('Award Date', etl.dateparser('%Y-%m-%d'))
    .cache()    
)

In [48]:
(
    data
        .cut('Amount Awarded', 'Funding Org:Name', 'Grant Programme:Title', 'Award Date')
        .convertnumbers()
        .aggregate(
            ('Funding Org:Name', 'Grant Programme:Title'),
            {
                'Grants': len,
                'Funding': ('Amount Awarded', sum),
                'Earliest Award Date': ('Award Date', min),
                'Latest Award Date': ('Award Date', max),
            }
        )
        .addfield('Average Grant', lambda r: r.Funding / r.Grants)
        .convert(('Funding', 'Average Grant'), round)
        .sort('Grants', reverse=True)
).tocsv(OUT / 'by_programme.csv')

In [52]:
data.cut(
    'Award Date', 'Amount Awarded'
).convertnumbers().convert(
    'Award Date', lambda d: d.replace(day=1)
).aggregate(
    'Award Date',
    {
        'Number': len,
        'Amount Awarded': ('Amount Awarded', sum),
    }
).convert(
    'Amount Awarded', round
).tocsv(
    OUT / 'summary_by_month.csv'
)